In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

# EDA

In [ ]:
df.columns

In [ ]:
for col in df.columns:
    if df[col].dtypes == object:
        print('------')
        print(df[col].describe())

In [ ]:
col_less_5 = []
print("Less than 5% missing values:\n")
for col in df.columns:
    percen_NaN = df[col].isnull().sum()/df.shape[0]*100   
    # Columns with more than 5% of NaN 
    if percen_NaN < 5:
        print(f'{col} : {df[col].isnull().sum()/df.shape[0]*100}%')
        col_less_5.append(col)
print("-------------")
print("\nMore than 5% missing values:\n")
for col in df.columns:
    percen_NaN = df[col].isnull().sum()/df.shape[0]*100   
    # Columns with more than 5% of NaN 
    if percen_NaN >= 5:
        print(f'{col} : {df[col].isnull().sum()/df.shape[0]*100}%')
        pass

In [ ]:
df[["Cloud9am","Cloud3pm"]]

# First with NaN < 5% 

In [ ]:
df_2 = df[col_less_5].dropna()
df_2[col_less_5].isna().sum(axis=0)

In [ ]:
print(f'{round((df_2[col_less_5].dropna().shape[0] - df[col_less_5].shape[0]) / df[col_less_5].shape[0] * 100,2)}%')

On a perdu 7.47% de l'information total mais toutes nos valeurs sont goods normalement

In [ ]:
df_2[col_less_5].info()

In [ ]:
#df_2[['RainToday','RainTomorrow']].value_counts()
df_2['RainToday'] = df_2['RainToday'].map(lambda x : 0 if x == "No" else 1)
df_2['RainTomorrow'] = df_2['RainTomorrow'].map(lambda x : 0 if x == "No" else 1)

In [ ]:
df_2.drop(['WindDir3pm'], axis = 1,inplace = True)

In [ ]:
df_2.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = df_2[df_2.columns[2:-1]].copy()
df_cr = (X - X.mean())/X.std()
df_cr['RainToday'] = df_2['RainToday']
df_cr['RainTomorrow'] = df_2['RainTomorrow']
df_cr = df_cr.reset_index()

In [ ]:
df_cr.head(2)

In [ ]:
col_interesting = ["MinTemp","MaxTemp","Humidity9am","Humidity3pm","Temp9am","Temp3pm","RainTomorrow"]

In [ ]:
df_cr[["RainToday","RainTomorrow"]].value_counts()

In [ ]:
import seaborn as sns
sns.pairplot(df_cr[col_interesting], hue="RainTomorrow")

# K-NN

'Humidity3pm','MinTemp', 'MaxTemp'

In [ ]:
y = df_cr["RainTomorrow"]
col_features = ["MinTemp","MaxTemp","Humidity9am","Humidity3pm","Temp9am","Temp3pm","RainToday"]
X = df_cr[col_features]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21, stratify=y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(X_train, y_train)
prediction = knn.predict(X_test)
knn.score(X_train,y_train)*100

In [ ]:
#Précision 
sum(prediction == y_test)/len(y_test)*100

# Test du best n_neighbors

In [ ]:
from sklearn.metrics import confusion_matrix
n_neighbors_list = range(1,21)
Testing_Accuracy = []
Training_Accuracy = []
for n_i in n_neighbors_list:
    knn = KNeighborsClassifier(n_neighbors= n_i)
    knn.fit(X_train, y_train)
    prediction = knn.predict(X_test)
    print(f'--\n{n_i} neighbors')
    print(confusion_matrix(y_test,prediction)/len(y_test)*100)
    Training_Accuracy.append(knn.score(X_train,y_train)*100)
    Testing_Accuracy.append(sum(prediction == y_test)/len(y_test)*100)

In [ ]:
print(Training_Accuracy)
print('---')
print(Testing_Accuracy)

In [ ]:
y_train.value_counts()[1]/y_train.value_counts()[0]

In [ ]:
Result_knn = pd.DataFrame({'n' : n_neighbors_list,'Training_Accuracy' : Training_Accuracy,'Testing_Accuracy' : Testing_Accuracy})
sns.lineplot(x = "n", 
             y = "Training_Accuracy",
             palette = "Set2",
             label = "Training",
             linewidth = 3,
             data = Result_knn)
sns.lineplot(x = "n", 
             y = "Testing_Accuracy", 
             palette = "Set2",
             label = "Testing",
             linewidth = 3,
             data = Result_knn)
plt.legend()
plt.show()

# ACP

In [ ]:
df_acp = df_2[df_2.columns[2:-1]]
df_acp.head(2)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
Z_acp = sc.fit_transform(df_acp)
#moyenne
print(np.mean(Z_acp,axis=0))
#écart-type
print(np.std(Z_acp,axis=0,ddof=0))

In [ ]:
from sklearn.decomposition import PCA
acp = PCA(svd_solver='full')
#calculs
coord = acp.fit_transform(Z_acp)
n = acp.n_components_
print(acp.n_components_) 

In [ ]:
#valeur corrigée
eigval = (Z_acp.shape[0]-1)/Z_acp.shape[0]*acp.explained_variance_
print(eigval)

In [ ]:
#proportion de variance expliquée
print(acp.explained_variance_ratio_)

In [ ]:
import matplotlib.pyplot as plt
#scree plot
plt.plot(np.arange(1,Z_acp.shape[1]+1),eigval)
plt.title("Scree plot")
plt.ylabel("Eigen values")
plt.xlabel("Factor number")
plt.arrow(x=4,y=3,dx=-1.75,dy=-0.88,width=0.04,head_starts_at_zero=True,color = 'red')
plt.arrow(x=6,y=2,dx=-1.75,dy=-0.88,width=0.04,head_starts_at_zero=True,color = 'green')
plt.text(4,3,'Elbow 1',color='red')
plt.text(6,2,'Elbow 2',color='green')
plt.show()

In [ ]:
#cumul de variance expliquée
plt.plot(np.arange(1,Z_acp.shape[1]+1),np.cumsum(acp.explained_variance_ratio_))
plt.title("Explained variance vs. # of factors")
plt.ylabel("Cumsum explained variance ratio")
plt.xlabel("Factor number")
plt.axvline(4,color='red')
plt.axhline(0.86,color='red')
plt.text(1,0.88,'86%',color='red')
plt.show()

On va prend k = 4  
k étant le nombre d'axes 

In [ ]:
ratio = Z_acp.shape[0]/Z_acp.shape[1]
#Une bonne ACP est si ratio > 3, alors c'est plutôt une bonne ACP 
print(ratio)

In [ ]:
#seuils pour test des bâtons brisés
bs = 1/np.arange(Z_acp.shape[1],0,-1)
bs = np.cumsum(bs)
bs = bs[::-1]

#test des bâtons brisés
print(pd.DataFrame({'Val.Propre':eigval,'Seuils':bs}))

In [ ]:
acp_test_color = df_2[df_2.columns[2:]]
acp_test_color
#modalités de la variable qualitative
modalites = np.unique(acp_test_color["RainTomorrow"])
print(modalites)

In [ ]:
for c in range(len(modalites)):
    print(c)
    print(len(acp_test_color.query(f"(RainTomorrow  ==  {modalites[c]})")))

In [ ]:
import seaborn as sns
#pour chaque modalité de la var. illustrative
for c in range(len(modalites)):
    #numéro des individus concernés
    numero = acp_test_color.query(f"(RainToday  ==  {modalites[c]})")

test2 = pd.DataFrame({'X':coord[:,0],'Y':coord[:,1],'RainToday': df_2["RainToday"]})

plt.subplots(figsize=(12,12))
sns.scatterplot(x = 'X', y = 'Y',hue="RainToday",alpha=0.3,
                data = test2)

In [ ]:
import seaborn as sns
#pour chaque modalité de la var. illustrative
for c in range(len(modalites)):
    #numéro des individus concernés
    numero = acp_test_color.query(f"(RainTomorrow  ==  {modalites[c]})")

test = pd.DataFrame({'X':coord[:,0],'Y':coord[:,1],'RainTomorrow': acp_test_color["RainTomorrow"]})

plt.subplots(figsize=(12,12))
sns.scatterplot(x = 'X', y = 'Y',hue="RainTomorrow",alpha=0.3,
                data = test)

In [ ]:
import plotly.express as px
test_3D = pd.DataFrame({'X':coord[:,0],'Y':coord[:,1],'Z':coord[:,2],'RainTomorrow': acp_test_color["RainTomorrow"]})
fig = px.scatter_3d(test_3D, x='X', y='Y', z='Z',color = 'RainTomorrow')
fig.show()

**Séparation trop marqué, donc trop grosse importance de la variable Rain Today**

In [ ]:
# Question 1.iii
fig, ax = plt.subplots(1,2, figsize=[20,5])
Rain_0 = test2[test2["RainToday"]  ==  0]
Rain_1 = test2[test2["RainToday"]  ==  1]

sns.scatterplot(ax = ax[0],x = 'X',
                y = 'Y',
                data = Rain_0)
sns.scatterplot(ax = ax[1],x = 'X',
                y = 'Y', 
                data = Rain_1,color='red')

ax[0].set_xlim(-10,10)
ax[0].set_ylim(-3,25)

ax[1].set_xlim(-10,10)
ax[1].set_ylim(-3,25)

In [ ]:
#contribution des individus dans l'inertie totale
di = np.sum(Z_acp**2,axis=1)
print(pd.DataFrame({'d_i':di}))

In [ ]:
#qualité de représentation des individus - COS2
cos2 = coord**2
for j in range(Z_acp.shape[1]):
    cos2[:,j] = cos2[:,j]/di
print(pd.DataFrame({'COS2_1':cos2[:,0],'COS2_2':cos2[:,1]}))

In [ ]:
#vérifions la théorie - somme en ligne des cos2 = 1
print(np.sum(cos2,axis=1).sum())

# Représentation des variables

In [ ]:
#racine carrée des valeurs propres
sqrt_eigval = np.sqrt(eigval)

In [ ]:
#corrélation des variables avec les axes
p = Z_acp.shape[1]
k = 4
corvar = np.zeros((p,p))
for k in range(p):
    corvar[:,k] = acp.components_[k,:] * sqrt_eigval[k]

#afficher la matrice des corrélations variables x facteurs 
print(corvar)

In [ ]:
#on affiche pour les deux premiers axes
pd.DataFrame({'COR_1':corvar[:,0],'COR_2':corvar[:,1]})

In [ ]:
#cercle des corrélations
fig, axes = plt.subplots(figsize=(8,8))
axes.set_xlim(-1,1)
axes.set_ylim(-1,1)
#affichage des étiquettes (noms des variables)
for j in range(p):
    plt.annotate(df_2[df_2.columns[2:-1]].columns[j],(corvar[j,0],corvar[j,1]))
    plt.scatter(corvar[j,0],corvar[j,1],color='blue')

# ajouter les axes
plt.plot([-1,1],[0,0],color='silver',linestyle='-',linewidth=1)
plt.plot([0,0],[-1,1],color='silver',linestyle='-',linewidth=1)

#ajouter un cercle
cercle = plt.Circle((0,0),1,color='blue',fill=False)
axes.add_artist(cercle)
#affichage
plt.show()